# Atomes pythoniques - une démo de la simplicité des classes construites avec dataclass

In [ ]:
import os
import re
import pandas as pd
from dataclasses import dataclass

Nous allons nous amuser à créer des objets personnalisés `Atom` et à les structurer de différentes manières.

Pour référence

![une table périodique des éléments](https://upload.wikimedia.org/wikipedia/commons/thumb/9/91/Tableau_p%C3%A9riodique_des_%C3%A9l%C3%A9ments.svg/1920px-Tableau_p%C3%A9riodique_des_%C3%A9l%C3%A9ments.svg.png)

## récupération et lecture des données

In [ ]:
periodic_table_url = "https://inventwithpython.com/periodictable.csv"
periodic_table_filename = "periodic_table.csv"

In [ ]:
# téléchargement du fichier CSV et enregistrement dans le même sous dossier. Nécessite curl.
os.system(f"curl -sS {periodic_table_url} > {periodic_table_filename}")

In [ ]:
def get_table():
    """Fonction qui charge le fichier CSV et renvoie une DataFrame 
    avec uniquement les colonnes qui nous intéressent."""
    
    # on charge le fichier, sans header car les entêtes de colonnes sont manquants
    table = pd.read_csv(periodic_table_filename, header=None)
    # liste de noms des colonnes
    columns = [
        "Atomic Number",
        "Symbol",
        "Element",
        "Origin of name",
        "Group",
        "Period",
        "Atomic_weight",
        "Density",
        "Melting point",
        "Boiling point",
        "Specific heat capacity",
        "Electronegativity",
        "Abundance in earth's crust",
    ]

    # ajout des noms de colonne à la DataFrame
    table.columns = columns

    # Sélection des colonnes qui nous intéressent en réalité
    selection = ["Atomic Number", "Symbol", "Element", "Group", "Period", "Atomic_weight"]
    table = table[selection]
    
    # retirer tous les nombres romains entre crochets issus du copier coller de wikipedia dans les poids
    clean_up_roman = lambda value: re.sub(r'\[(I|V|X)+\]', '', value)
    table.Atomic_weight = table.Atomic_weight.apply(clean_up_roman)

    # changer la colonne group de float à int
    table.Group = table.Group.astype("Int32")

    # régler l'index sur le numéro atomique et renvoyer la DataFrame
    return table.set_index(columns[0])

In [ ]:
table = get_table()

In [ ]:
table.head()  # juste pour vérifier les 5 premières lignes

In [ ]:
table[table.Symbol.str.startswith("C")] # atomes dont le symbole commence par C

## Création d'une classe pour stocker les informations d'un atome.
Ici nous allons utiliser le décorateur `dataclass` que nous avons importé au début pour simplifier grandement la définition de cette classe.

Avec `dataclass`, nous n'avons qu'à lister les attributs que l'on veut, et leur type. Les méthodes magiques `__init__()`, `__repr__()` et `__eq__()` vont être automatiquement construites pour nous.

L'argument `frozen=True` dans le décorateur fera en sorte que nos atomes soient immuables une fois créés, ce qui évitera des réassignations de valeurs hasardeuses post instanciation.

Enfin l'argument `order=True` indique que nos atomes seront automatiquement classables, ils seront comparés dans l'ordre de leurs attributs (ici `number` est le premier attribut, et comme chaque valeur est unique, ce sera effectivement le seul critère pour classer une liste d'atomes.). Ainsi on évite d'avoir à implémenter des méthodes de comparaison classiques comme `__lt__()`, `__le__()`, `__gt__()`, et `__ge__()`, c'est fait automatiquement. Magie pure.

In [ ]:
@dataclass(frozen=True, order=True)
class Atom:
    """Une micro classe pour représenter un atome."""
    number: int
    name: str
    symbol: str
    group: int
    period: int
    weight: float

In [ ]:
C = Atom(6, "Carbon", "C", 14, 2, 0.0)  # simple test d'instanciation de la classe

In [ ]:
C  # la méthode __repr__() automatique fonctionne, merci dataclass

## Création d'une instance d'atome pour chaque atome listé dans notre table, dans une liste apelée `atoms`.

In [ ]:
atoms = [
    Atom(number, name, symbol, group, period, weight)
    for number, name, symbol, group, period, weight in zip(
        table.index, table.Element, table.Symbol, table.Group, table.Period, table.Atomic_weight
    )
]

In [ ]:
atoms[50:60] # petit contrôle sure une tranche de 10 atomes

À ce stade on réalise que certains atomes n’ont pas d'information de groupe.

## Création d'un dictionnaire des différents groupes
Nous allons créer un dictionnaire avec en clé les numéros de groupe et en valeurs une liste des atomes correspondants, mais nous devons traiter les atomes non groupés à part.

Commençons par isoler ceux qui ont un groupe différent de `pd.NA`.

In [ ]:
grouped = [atom for atom in atoms if not atom.group is pd.NA]

In [ ]:
len(atoms) - len(grouped)

Il y a 28 atomes sans groupe apparemment.

On crée notre dictionnaire de la forme `{numéro_de_groupe : [atomes du groupe]}`, avec cette dictionary comprehension.

In [ ]:
groups = {
    n: [atom for atom in grouped if atom.group == n] # format des clés et des valeurs
    for n in range(1, table.Group.max() + 1) # listes des valeurs : entiers entre 1 et le plus grand numéro de groupe +1 car range s'arrête toujours avant le max.
}

In [ ]:
groups.get(1) # on regarde rapidement le premier groupe, une liste d'atomes ayant group==1

In [ ]:
# pour faire une liste des atomes sans groupe, on peut faire
# la différence entre l'ensemble des atomes et l'ensemble des atomes groupés, et reconstruire une liste
ungrouped = list(set(atoms) - set(grouped))
len(ungrouped)

In [ ]:
# alternative, avec une list-comprehension
ungrouped = [atom for atom in atoms if atom.group is pd.NA]
len(ungrouped)

In [ ]:
ungrouped

In [ ]:
# finalement on les ajoute avec une nouvelle clé dans notre dictionnaire des groupes
groups[0] = ungrouped

In [ ]:
groups.get(2) # maintenant c'est très facile d'obtenir un groupe dans notre dictionnaire

In [ ]:
groups.get(12)

## Création d'un dictionnaire des périodes

Dans le même esprit que notre dictionnaire des groupes, faisons en un avec les périodes

In [ ]:
# test pour voir s'il y a des éléments sans période dans notre table initiale 
table[table.Period.isna()]

Bonne nouvelle, tous nos atomes ont une période dans le jeu de données initial, et donc dans tous les objets de classe `Atom` que nous avons créés. La création du dictionnaire de forme `{période: [atomes avec cette période]}` se fait très simplement :

In [ ]:
periods = {
    p: [atom for atom in atoms if atom.period == p] # format de notre paire clé : valeur. 
    for p in range(1, table.Period.max() +1) # on itère sur la liste des valeurs possibles pour les périodes
}

Maintenant c'est très pratique d'attraper tous les atomes d'une période donnée !

In [ ]:
periods.get(1)

In [ ]:
periods.get(2)